In [ ]:
__author__ = 'Knut Olsen <kolsen@noao.edu>, Alice Jacques <alice.jacques@noao.edu>' 
__version__ = '20200915' # yyyymmdd;
__datasets__ = ['nsc_dr1','svc_stripe82']  
__keywords__ = ['image cutout','SIA','tutorial']

# How to use the Simple Image Access (SIA) Service to make image cutouts

*Knut Olsen & NOIRLab Astro Data Lab Team*

### Table of contents
* [Goals & notebook summary](#goals)
* [Disclaimer & Attribution](#attribution)
* [Imports & setup](#import)
* [What is the Simple Image Access (SIA) service?](#chapter1)
* [Example usage](#chapter2)
* [Making the process easier with functions](#chapter3)

<a class="anchor" id="goals"></a>
# Goals
Show how to use the Simple Image Access (SIA) Service API to make image cutouts.

# Summary
In this notebook, we describe and demonstrate the SIA Service API and use it to make an example set of image cutouts.  We list all of the available image collections available through SIA services, and then select one to retrieve an image cutout at a specific RA, Dec on the sky.  We define a function to identify the deepest image available at a specific band and position, retrieve g, r, and i band image cutouts at that position, and make a three-band color image from the cutouts.

<a class="anchor" id="attribution"></a>
# Disclaimer & Attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: http://datalab.noao.edu/disclaimers.php

<a class="anchor" id="import"></a>
# Imports and setup

In [ ]:
# 3rd party 
import numpy as np
from numpy.core.defchararray import startswith
import pylab as plt
import matplotlib
%matplotlib inline

# SIA library and utilities for accessing FITS images and making three-color images
from pyvo.dal import sia
from astropy.utils.data import download_file
from astropy.io import fits
from astropy.wcs import WCS
from astropy.visualization import make_lupton_rgb

# Data Lab
from dl import authClient as ac, queryClient as qc, storeClient as sc

# Quiet the Astropy warnings
import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

# Authentication
Much of the functionality of Data Lab can be accessed without explicitly logging in (the service then uses an anonymous login). But some capacities, for instance saving the results of your queries to your virtual storage space, require a login (i.e. you will need a registered user account).

If you need to log in to Data Lab, issue this command, and respond according to the instructions:

In [ ]:
#ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))
ac.whoAmI()

<a class="anchor" id="chapter1"></a>
# What is the Simple Image Access (SIA) service?
The SIA service is a tool to identify the set of images in an image collection that overlap the position at which an image cutout is desired.  It returns a table of potential images containing information (metadata) typically found in the image headers (filter, exposure time, time of observation, etc.) as well as a URL pointing to the image cutout itself.  Getting image cutouts is thus a three-step process:

1) choose an image collection  
2) call the SIA service for that collection to retrieve information about images that overlap a position of interest  
3) retrieve the image cutouts by accessing the URLs for the selected images in the table of information  

Choosing a specific image collection is done by selecting the URL that points to that collection.  There are lots of available collections, including images from the Dark Energy Survey, Legacy Survey, and the entire NOIRLab Astro Data Archive. For more background on the SIA protocol, see the [IVOA document](http://www.ivoa.net/documents/SIA/) that describes it.

Here we list the Simple Image Access (SIA) service endpoints for different image collections:

In [ ]:
# The default endpoint points to the entire public NOIRLab Astro Data Archive
DEF_ACCESS_URL = "http://datalab.noao.edu/sia"
svc = sia.SIAService(DEF_ACCESS_URL)

# The NOIRLab Astro Data Archive is the default endpoint.  It contains all public data in the archive, including
# raw and processed images, data quality masks, etc.
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/nsa"
svc_nsa = sia.SIAService(DEF_ACCESS_URL)

# The subset of images from the NOIRLab Astro Data Archive that overlap with SDSS Stripe 82:
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_nsa"
svc_stripe82 = sia.SIAService(DEF_ACCESS_URL)
# Other targeted image subsets that overlap with SDSS Stripe 82:
# To use these, call svc = sia.SIAService(DEF_ACCESS_URL)
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_calibrated"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_coadd"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_decaps_dr1"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_des_dr1"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_des_sva1"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_des_y1"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_des_y2"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_des_y3"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_des_y4"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_ls_dr3"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_ls_dr5"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_ls_dr6"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_ls_dr7"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_ls_dr8"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_raw"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_smash_dr1"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_smash_dr2"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_splus_dr1"
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/stripe82_splus_edr"

# The subset of instrument calibrated images from the NOIRLab Astro Data Archive:
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/calibrated_all"
svc_cal = sia.SIAService(DEF_ACCESS_URL)
# Other collections of instrument calibrated images for specific surveys:
# To use these, call svc = sia.SIAService(DEF_ACCESS_URL)
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/calibrated_des_y1" # DES Year 1 calibrated images
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/calibrated_des_y2" # DES Year 2 calibrated images
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/calibrated_des_y3" # DES Year 3 calibrated images
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/calibrated_des_y4" # DES Year 4 calibrated images
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/calibrated_nsa" # NSA calibrated images
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/calibrated_smash_dr1" # SMASH DR1 calibrated images
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/calibrated_smash_dr2" # SMASH DR2 calibrated images

# The subset of stacked calibrated images from the NOIRLab Astro Data Archive:
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/coadd_all"
svc_coadd = sia.SIAService(DEF_ACCESS_URL)
# Other collections of stacked images for specific surveys:
# To use these, call svc = sia.SIAService(DEF_ACCESS_URL)
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/coadd_decaps_dr1" # DECaPS DR1 stacks
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/coadd_des_dr1" # DES DR1 stacks
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/coadd_des_sva1" # DES SVA1 stacks
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/coadd_ls_dr3" # Legacy Survey DR3 stacks
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/coadd_ls_dr4" # Legacy Survey DR4 stacks
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/coadd_ls_dr5" # Legacy Survey DR5 stacks
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/coadd_ls_dr6" # Legacy Survey DR6 stacks
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/coadd_ls_dr7" # Legacy Survey DR7 stacks
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/coadd_ls_dr8" # Legacy Survey DR8 stacks
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/coadd_smash_dr1" # SMASH DR1 stacks
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/coadd_splus_dr1" # S-PLUS DR1 stacks

# The subset of raw uncalibrated images from the NOIRLab Astro Data Archive
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/raw_all"
svc_raw = sia.SIAService(DEF_ACCESS_URL)

# DECam images from the DECam Plane Survey DR1
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/decaps_dr1"
svc_decaps_dr1 = sia.SIAService(DEF_ACCESS_URL)

# The Dark Energy Survey DR1 image release contains images from the first three years of DES
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/des_dr1"
svc_des_dr1 = sia.SIAService(DEF_ACCESS_URL)

# The Dark Energy Survey SV image release contains images from the DES SV period
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/des_sva1"
svc_des_sva1 = sia.SIAService(DEF_ACCESS_URL)

# The subset of Dark Energy Survey DR1 images obtained in the first year of DES
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/des_y1"
svc_des_y1 = sia.SIAService(DEF_ACCESS_URL)

# The subset of Dark Energy Survey DR1 images obtained in the second year of DES
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/des_y2"
svc_des_y2 = sia.SIAService(DEF_ACCESS_URL)

# The subset of Dark Energy Survey DR1 images obtained in the third year of DES
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/des_y3"
svc_des_y3 = sia.SIAService(DEF_ACCESS_URL)

# The subset of Dark Energy Survey DR1 images obtained in the fourth year of DES
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/des_y4"
svc_des_y4 = sia.SIAService(DEF_ACCESS_URL)

# Images from GOGREEN DR1 
DEF_ACCESS_URL = "https://datalab.noao.edu/sia/gogreen_dr1"
svc_gogreen_dr1 = sia.SIAService(DEF_ACCESS_URL)

# DECam images from Legacy Survey DR3
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/ls_dr3"
svc_ls_dr3 = sia.SIAService(DEF_ACCESS_URL)

# Mosaic-3 and Bok images from Legacy Survey DR4
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/ls_dr4"
svc_ls_dr4 = sia.SIAService(DEF_ACCESS_URL)

# DECam images from Legacy Survey DR5
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/ls_dr5"
svc_ls_dr5 = sia.SIAService(DEF_ACCESS_URL)

# Mosaic-3 and Bok images from Legacy Survey DR6
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/ls_dr6"
svc_ls_dr6 = sia.SIAService(DEF_ACCESS_URL)

# DECam images from Legacy Survey DR7
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/ls_dr7"
svc_ls_dr7 = sia.SIAService(DEF_ACCESS_URL)

# Mosaic-3 and Bok images from Legacy Survey DR8
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/ls_dr8"
svc_ls_dr8 = sia.SIAService(DEF_ACCESS_URL)

# DECam images from the SMASH Survey DR1 release
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/smash_dr1"
svc_smash_dr1 = sia.SIAService(DEF_ACCESS_URL)

# DECam images from the SMASH Survey DR2 release
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/smash_dr2"
svc_smash_dr2 = sia.SIAService(DEF_ACCESS_URL)

# T80S telescope images from the S-PLUS DR1 release
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/splus_dr1"
svc_splus_dr1 = sia.SIAService(DEF_ACCESS_URL)

# T80S telescope images from the S-PLUS EDR release
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/splus_edr"
svc_splus_edr = sia.SIAService(DEF_ACCESS_URL)

<a class="anchor" id="chapter2"></a>
# Example usage
We will pick a location in Stripe 82 and use the Stripe 82-specific SIA service to retrieve an image cutout 6x6 arcmin in size.  We use the search method of the SIA service object to perform the search, and return the result as an astropy Table object.  

The search method takes the following as arguments:
* a tuple containing (ra,dec)  
* a second tuple with the size of the cutout in each direction (field of view or fov)

The search works in RA and Dec in degrees, and does not account for the convergence of lines of RA toward the poles, so we divide by a `cos(dec)` term in the RA size argument.

In [ ]:
ra = 335.0
dec = 0.0
fov = 0.1

imgTable = svc_stripe82.search((ra,dec), (fov/np.cos(dec*np.pi/180), fov), verbosity=2).to_table()

The result is a table of metadata for images that overlap the search position:

In [ ]:
imgTable

The column `access_url` contains the link to retrieve the image cutout.  We can filter the table on any of the columns to narrow down the list to just the cutouts we want.  Here we'll select stacked images (`proctype=Stack` and `prodtype=image`) observed in the g filter (`obs_bandpass` starts with the the letter 'g'):

In [ ]:
sel = (imgTable['proctype'].astype(str)=='Stack') & (imgTable['prodtype'].astype(str)=='image') & \
    (startswith(imgTable['obs_bandpass'].astype(str),'g'))

We'll pick a row from the filtered table and retrieve the image.

In [ ]:
row = imgTable[sel][1]
url = row['access_url'].decode() # get the download URL
filename = download_file(url,cache=True,show_progress=False,timeout=120)
hdu = fits.open(filename)[0]
image = hdu.data
hdr = hdu.header
wcs = WCS(hdr) # WCS info from header to convert pixel coords to world coords

Then we'll display the image.  Note that the image might not be square for a variety of reasons, such as that the image cutout ran into a chip edge along one axis.

In [ ]:
print("Min pixel:", image.min(), "\nMax pixel:", image.max(),"\nImage size:",image.shape)
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(projection=wcs)
ax.imshow(image,origin='lower',cmap='bone',vmin=image.min(),vmax=image.min()+(image.max()-image.min())/170.)

### Catalog overlay
We can also query the database for catalog objects and overlay them easily on the image cutout.  In this example, we query around the central position for objects in the NOAO Source Catalog DR1 object table:

In [ ]:
sql = '''SELECT ra, dec, gmag 
            FROM nsc_dr1.object 
            WHERE q3c_radial_query(ra,dec,{0},{1},{2})
            AND gerr < 0.1 AND gmag < 22
'''.format(ra, dec, fov/2)
df = qc.query(sql=sql, fmt='pandas')

Here we'll use the object *g*-band magnitudes to control the sizes of the markers.

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1, projection=wcs)
ax.imshow(image, cmap='bone', vmin=image.min(), vmax=image.min()+(image.max()-image.min())/170.)
ax.scatter(df['ra'], df['dec'], transform=ax.get_transform('icrs'), s=1000./(df['gmag']-12), 
           edgecolor='green', facecolor='none', linewidths=3)

## Save image to VOSpace
If you're not intending to display or work with the image in the notebook, but instead want to save the image as a file, the storeClient (`sc`) can copy the cutout URL directly to VOSpace if you are logged in to Data Lab.

Save the contents of the image cutout URL to VOSpace:

In [ ]:
sc.load('cutout.fits',url)

Let's make sure the image was saved to VOSpace by calling the `sc.ls` function specifying the name of the image:

In [ ]:
sc.ls('cutout.fits')

<a class="anchor" id="chapter3"></a>
# Making the process easier with functions
The example workflow above lends itself to writing a short function to access the SIA service, filtering the metadata table to narrow down the search result, and retrieving the cutout.  Here we define two functions: (1) a function to get the deepest stacked image for a given image collection and band, and (2) a function to plot the images.

In [ ]:
# A little function to download the deepest stacked images
#   adapted from R. Nikutta
def download_deepest_image(ra,dec,svc=sia.SIAService('http://datalab.noao.edu/sia'),fov=0.1,band='g'):
    imgTable = svc.search((ra,dec), (fov/np.cos(dec*np.pi/180), fov), verbosity=2).to_table()
    
    sel0 = startswith(imgTable['obs_bandpass'].astype(str),band)
    print("The full image list contains", len(imgTable[sel0]), "entries with bandpass="+band)

    sel = sel0 & ((imgTable['proctype'].astype(str)=='Stack') & (imgTable['prodtype'].astype(str)=='image'))
    Table = imgTable[sel]
    if (len(Table)>0):
        row = Table[np.argmax(Table['exptime'].data.data.astype('float'))] # pick image with longest exposure time
        url = row['access_url'].decode() # get the download URL
        print ('downloading deepest '+band+' image...')
        image = fits.getdata(download_file(url,cache=True,show_progress=False,timeout=120))

    else:
        print ('No image available.')
        image=None
        
    return image

In [ ]:
# Multi panel image plotter
def plot_images(images,geo=None,panelsize=4,bands=list('gri'),cmap=matplotlib.cm.gray_r):
    n = len(images)
    if geo is None: geo = (n,1)
        
    fig = plt.figure(figsize=(geo[0]*panelsize,geo[1]*panelsize))
    for j,img in enumerate(images):
        ax = fig.add_subplot(geo[1],geo[0],j+1)
        if img is not None:
            print("min pixel ("+bands[j]+" band):",img.min(),"max pixel ("+bands[j]+" band):",img.max())
            vmin = np.median(img)-2*np.std(img)
            vmax = np.median(img)+2*np.std(img)
            ax.imshow(img,origin='lower',interpolation='none',cmap=cmap,vmin=vmin, vmax=vmax)
            ax.set_title('%s band' % bands[j])
            ax.xaxis.set_visible(False)
            ax.yaxis.set_visible(False)

We run the function to get cutouts from deep stack in g, r, and i bands.

In [ ]:
band = 'g'
gimage = download_deepest_image(ra, dec, fov=0.1, band=band) # FOV in deg
band = 'r'
rimage = download_deepest_image(ra, dec, fov=0.1, band=band) # FOV in deg
band = 'i'
iimage = download_deepest_image(ra, dec, fov=0.1, band=band) # FOV in deg

Now we plot the image per band:

In [ ]:
images=[gimage,rimage,iimage]
plot_images(images)

And finally make a three-color image from the output:

In [ ]:
gimage2 = gimage-np.median(gimage)
rimage2 = rimage-np.median(rimage)
iimage2 = iimage-np.median(iimage)
img = make_lupton_rgb(iimage2, rimage2, gimage2, Q=10, stretch=30) # Q=asinh softening parameter

fig = plt.figure(figsize=[12,12])
ax = fig.add_subplot(1,1,1)

ax.imshow(img,origin='lower')
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)